In [1]:
import pandas as pd
import subprocess
import json

In [2]:
data = pd.read_parquet('../data/curated/Twitter_Cleaned/fully_cleaned_tweets.parquet')

In [3]:
data = data.drop(['x_cent', 'y_cent', 'p>n', 'p>nu', 'nu>n'], axis = 1)

Add Date

In [4]:
from datetime import timedelta, datetime

In [5]:
ELECTION_DATE = datetime(year = 2022, month = 5, day = 21)
START_DATE = datetime(year = 2022, month = 2, day = 10)
END_DATE = datetime(year = 2023, month = 6, day = 30)

In [6]:
date_week_map = dict()
date_ = ELECTION_DATE

switch = 1
i = 0
week = 1
while switch:

    date_ += timedelta(days = 1)

    date_week_map[date_] = week

    i += 1
    if i == 7:
        i = 0
        week += 1
        

    if date_ == END_DATE:
        switch = 0


switch = 1
i = 0
week = -1
date_ = ELECTION_DATE
while switch:


    date_week_map[date_] = week
    date_ -= timedelta(days = 1)

    i += 1
    if i == 7:
        i = 0
        week -= 1

    if date_ < START_DATE:
        switch = 0

In [7]:
date_week_map

{datetime.datetime(2022, 5, 22, 0, 0): 1,
 datetime.datetime(2022, 5, 23, 0, 0): 1,
 datetime.datetime(2022, 5, 24, 0, 0): 1,
 datetime.datetime(2022, 5, 25, 0, 0): 1,
 datetime.datetime(2022, 5, 26, 0, 0): 1,
 datetime.datetime(2022, 5, 27, 0, 0): 1,
 datetime.datetime(2022, 5, 28, 0, 0): 1,
 datetime.datetime(2022, 5, 29, 0, 0): 2,
 datetime.datetime(2022, 5, 30, 0, 0): 2,
 datetime.datetime(2022, 5, 31, 0, 0): 2,
 datetime.datetime(2022, 6, 1, 0, 0): 2,
 datetime.datetime(2022, 6, 2, 0, 0): 2,
 datetime.datetime(2022, 6, 3, 0, 0): 2,
 datetime.datetime(2022, 6, 4, 0, 0): 2,
 datetime.datetime(2022, 6, 5, 0, 0): 3,
 datetime.datetime(2022, 6, 6, 0, 0): 3,
 datetime.datetime(2022, 6, 7, 0, 0): 3,
 datetime.datetime(2022, 6, 8, 0, 0): 3,
 datetime.datetime(2022, 6, 9, 0, 0): 3,
 datetime.datetime(2022, 6, 10, 0, 0): 3,
 datetime.datetime(2022, 6, 11, 0, 0): 3,
 datetime.datetime(2022, 6, 12, 0, 0): 4,
 datetime.datetime(2022, 6, 13, 0, 0): 4,
 datetime.datetime(2022, 6, 14, 0, 0): 4,
 

In [8]:
def map_week_relateive_to_election(date):
    return date_week_map[date]

In [9]:
data['date'] = pd.to_datetime(data['date'])

In [10]:
data['week'] = data['date'].apply(map_week_relateive_to_election)

In [123]:
data.to_parquet('../data/curated/Twitter_Cleaned/fully_cleaned_tweets_with_week.parquet')

In [14]:
doc_list = []
for id, row in data.iterrows():

    row_json = {"_id": f"{row['_id']}", "author_id": f"{row['author_id']}", "date": f"{row['date']}", "text": row['text'], 
            "is_political": row['is_political'], "neg_score": row['neg_score'], "neu_score": row['neu_score'], 
            "pos_score": row['pos_score'], "compound_score": row['compound_score'], "sentiment": row['sentiment'], 
            "electorate": row['electorate'], "week": row['week']}
    doc_list.append(row_json)

out = {'docs': doc_list}

with open('../data/curated/Twitter_Cleaned/twitter_cdb_bulk.json', 'w') as f:
    json.dump(out, f)

In [26]:
with open('../data/curated/Twitter_Cleaned/twitter_cdb_bulk.json', 'r') as f:
    twitter_cdb_bulk = json.load(f)

In [24]:
subprocess.run(f'curl -X PUT http://172.26.133.251:5984/tweet_database -u group9_admin:group9_H1', shell=True)

{"ok":true}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    12  100    12    0     0    233      0 --:--:-- --:--:-- --:--:--   240


CompletedProcess(args='curl -X PUT http://172.26.133.251:5984/tweet_database -u group9_admin:group9_H1', returncode=0)

In [32]:
for i in range(len(twitter_cdb_bulk['docs'])//10000):
    out = {'docs': twitter_cdb_bulk['docs'][i*10000:(i+1)*10000]}
    json_data = json.dumps(out).encode('utf-8')
    subprocess.run(['curl', '-X', 'POST', f'http://172.26.133.251:5984/tweet_database/_bulk_docs', '--header', 'Content-Type: application/json', '--data-binary', '@-', '-u', 'group9_admin:group9_H1'], input=json_data)

out = {'docs': twitter_cdb_bulk['docs'][(i+1)*10000:len(twitter_cdb_bulk['docs'])]}
json_data = json.dumps(out).encode('utf-8')
subprocess.run(['curl', '-X', 'POST', f'http://172.26.133.251:5984/tweet_database/_bulk_docs', '--header', 'Content-Type: application/json', '--data-binary', '@-', '-u', 'group9_admin:group9_H1'], input=json_data)
